<a href="https://colab.research.google.com/github/SaraSaponaro/segmentation_program/blob/master/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1wgcrQHN-2wtuMknwJDLSzVN00olOO8Bc?authuser=1#scrollTo=6L0Zl9UsojSD)

In [0]:
from google.colab import files 
uploaded = files.upload() 

Saving feature_al_ML.txt to feature_al_ML.txt


In [0]:
import pandas as pd
#tutte: names=['filename ', 'classe', 'area ' ,'perimeter ' ,'circularity ', 'mu_NRL ' ,'std_NRL ', 'zero_crossing ' ,'max_axis ' ,'min_axis ', 'mu_VR', ' std_VR ' ,'RLE ',' convexity ',' mu_I ', 'std_I ' ,'kurtosis ','skewness'])
#our:names=['filename ', 'classe', 'area'  ,'circularity', 'mu_NRL ' ,'std_NRL ', 'mu_VR','RLE ',' convexity ','std_I ' ,'kurtosis ','skewness'])
#ref:filename, classe, area, circ ,mu_NRL, std_NRL, E, conv, istd, kurt, sk
mass = pd.read_table('feature_al_ML.txt', sep='\t', delim_whitespace=True, index_col=False, names=['filename ', 'classe', 'area'  ,'circularity', 'mu_NRL ' ,'std_NRL ', 'mu_VR','RLE ',' convexity ','std_I ' ,'kurtosis ','skewness'])
mass

#mass.columns
#mass['classe'].unique()
mass.groupby('classe').size()
#stack = mass.groupby(['classe'])['area'].count()
#stack


In [0]:
import seaborn as sns 
sns.pairplot(mass.loc[:,'classe':'skewness'], hue='classe')

In [0]:
# 2-fold cross validation
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import StratifiedKFold
import numpy
from matplotlib import pyplot
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt('feature_al_ML.txt', delimiter='\t', usecols=(1,2,3,4,5,6,7,8,9))
# split into input (X) and output (y) variables
X = dataset[:,1:]
Y = dataset[:,0]
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, Y):
    
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    #model.add(Dense(20, activation='relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(8, activation='relu'))
    #model.add(Dropout(0.1))
    
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    history=model.fit(X[train], Y[train],validation_data=(X[test], Y[test]), epochs=6, batch_size=7, verbose=0)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))


#aggiungi 
#from keras.utils import plot_model plot_model(model, to_file='model.png')
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['acc'], label='train')
pyplot.plot(history.history['val_acc'], label='test')
pyplot.legend()
pyplot.show()

#history.history


In [0]:
from sklearn.metrics import roc_curve, auc 
import pylab as plt 

y_pred_keras = model.predict(X).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(Y, y_pred_keras, pos_label=2)
auc_keras = auc(fpr_keras, tpr_keras)
print(auc_keras)
print('fpr',fpr_keras)
print('tpr',tpr_keras)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()



